In [49]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Merge, Embedding, Convolution1D,Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence


from features import *
from helper import *

In [2]:
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [3]:
import pickle
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [18]:
mocap_head = []

for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,6)):
        x_head = np.zeros((200,6))  
    x_head[np.isnan(x_head)]=0
    mocap_head.append( x_head )
    
mocap_head = np.array(mocap_head)
mocap_head.shape

(4936, 200, 6)

In [19]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

In [23]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(256, return_sequences=False, input_shape=(200, 6)))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

    return model

In [27]:
model = lstm_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 256)               269312    
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
activation_13 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 516       
_________________________________________________________________
activation_14 (Activation)   (None, 4)                 0         
Total params: 302,724
Trainable params: 302,724
Non-trainable params: 0
_________________________________________________________________


In [28]:
hist = model.fit(mocap_head, Y, 
                 batch_size=100, nb_epoch=25, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/25
3948/3948 [==============================] - 12s - loss: 1.3925 - acc: 0.3098 - val_loss: 1.3968 - val_acc: 0.3168
Epoch 2/25
3948/3948 [==============================] - 12s - loss: 1.3154 - acc: 0.3774 - val_loss: 1.3769 - val_acc: 0.3441
Epoch 3/25
3948/3948 [==============================] - 12s - loss: 1.2857 - acc: 0.4182 - val_loss: 1.3658 - val_acc: 0.3522
Epoch 4/25
3948/3948 [==============================] - 12s - loss: 1.2641 - acc: 0.4243 - val_loss: 1.3651 - val_acc: 0.3603
Epoch 5/25
3948/3948 [==============================] - 12s - loss: 1.2483 - acc: 0.4422 - val_loss: 1.3580 - val_acc: 0.3704
Epoch 6/25
3948/3948 [==============================] - 12s - loss: 1.2370 - acc: 0.4524 - val_loss: 1.3603 - val_acc: 0.3725
Epoch 7/25
3948/3948 [==============================] - 12s - loss: 1.2292 - acc: 0.4602 - val_loss: 1.3577 - val_acc: 0.3775
Epoch 8/25
3948/3948 [==============================] - 12s - loss: 1.2

In [29]:
def dense_model():
    model = Sequential()
    model.add(Flatten(input_shape=(200, 6)))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

    return model

In [30]:
model = dense_model()
model.summary()

hist = model.fit(mocap_head, Y, 
                 batch_size=100, nb_epoch=30, verbose=1, shuffle = True, 
                 validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               307456    
_________________________________________________________________
activation_15 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 1028      
_________________________________________________________________
activation_16 (Activation)   (None, 4)                 0         
Total params: 308,484
Trainable params: 308,484
Non-trainable params: 0
_________________________________________________________________
Train on 3948 samples, validate on 988 samples
Epoch 1/30


/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


3948/3948 [==============================] - 0s - loss: 10.9097 - acc: 0.3040 - val_loss: 9.9780 - val_acc: 0.3745
Epoch 2/30
3948/3948 [==============================] - 0s - loss: 10.5535 - acc: 0.3364 - val_loss: 10.7542 - val_acc: 0.3138
Epoch 3/30
3948/3948 [==============================] - 0s - loss: 10.5025 - acc: 0.3336 - val_loss: 9.8508 - val_acc: 0.3684
Epoch 4/30
3948/3948 [==============================] - 0s - loss: 10.1769 - acc: 0.3526 - val_loss: 10.2918 - val_acc: 0.3411
Epoch 5/30
3948/3948 [==============================] - 0s - loss: 10.0796 - acc: 0.3584 - val_loss: 9.7977 - val_acc: 0.3755
Epoch 6/30
3948/3948 [==============================] - 0s - loss: 9.9483 - acc: 0.3683 - val_loss: 10.0228 - val_acc: 0.3593
Epoch 7/30
3948/3948 [==============================] - 0s - loss: 10.0831 - acc: 0.3627 - val_loss: 10.1249 - val_acc: 0.3553
Epoch 8/30
3948/3948 [==============================] - 0s - loss: 10.0087 - acc: 0.3645 - val_loss: 9.4695 - val_acc: 0.3907


In [31]:
mocap_hand = []

for ses_mod in data2:
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,18)):
        x_hand = np.zeros((200,18))  
    x_hand[np.isnan(x_hand)]=0
    mocap_hand.append( x_hand )
    
mocap_hand = np.array(mocap_hand)
mocap_hand.shape

(4936, 200, 18)

In [34]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(256, return_sequences=False, input_shape=(200, 18)))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

    return model

In [35]:
model = lstm_model()
model.summary()

hist = model.fit(mocap_hand, Y, 
                 batch_size=100, nb_epoch=30, verbose=1, shuffle = True, 
                 validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 256)               281600    
_________________________________________________________________
dense_19 (Dense)             (None, 128)               32896     
_________________________________________________________________
activation_19 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 516       
_________________________________________________________________
activation_20 (Activation)   (None, 4)                 0         
Total params: 315,012
Trainable params: 315,012
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 12s - loss: 1.3602 - acc: 0.3392 - val_loss: 1.4126 - val_acc: 0.2874
Epoch 2/30
3948/3948 [==============================] - 12s - loss: 1.2965 - acc: 0.4012 - val_loss: 1.4367 - val_acc: 0.3219
Epoch 3/30
3948/3948 [==============================] - 12s - loss: 1.2719 - acc: 0.4504 - val_loss: 1.4458 - val_acc: 0.3117
Epoch 4/30
3948/3948 [==============================] - 12s - loss: 1.2541 - acc: 0.4650 - val_loss: 1.4950 - val_acc: 0.2702
Epoch 5/30
3948/3948 [==============================] - 12s - loss: 1.2515 - acc: 0.4400 - val_loss: 1.4775 - val_acc: 0.2905
Epoch 6/30
3948/3948 [==============================] - 12s - loss: 1.2361 - acc: 0.4635 - val_loss: 1.4288 - val_acc: 0.2864
Epoch 7/30
3948/3948 [==============================] - 12s - loss: 1.2185 - acc: 0.4813 - val_loss: 1.4356 - val_acc: 0.3279
Epoch 8/30
3948/3948 [==============================] - 12s - loss: 1.2

In [37]:
def dense_model():
    model = Sequential()
    model.add(Flatten(input_shape=(200, 18)))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

    return model

In [38]:
model = dense_model()
model.summary()

hist = model.fit(mocap_hand, Y, 
                 batch_size=100, nb_epoch=30, verbose=1, shuffle = True, 
                 validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 3600)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 256)               921856    
_________________________________________________________________
activation_21 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 1028      
_________________________________________________________________
activation_22 (Activation)   (None, 4)                 0         
Total params: 922,884
Trainable params: 922,884
Non-trainable params: 0
_________________________________________________________________
Train on 3948 samples, validate on 988 samples
Epoch 1/30


/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


3948/3948 [==============================] - 0s - loss: 10.6613 - acc: 0.3381 - val_loss: 10.1337 - val_acc: 0.3694
Epoch 2/30
3948/3948 [==============================] - 0s - loss: 10.6556 - acc: 0.3389 - val_loss: 10.1337 - val_acc: 0.3694
Epoch 3/30
3948/3948 [==============================] - 0s - loss: 10.6556 - acc: 0.3389 - val_loss: 10.1337 - val_acc: 0.3694
Epoch 4/30
3948/3948 [==============================] - 0s - loss: 10.6556 - acc: 0.3389 - val_loss: 10.1337 - val_acc: 0.3694
Epoch 5/30
3948/3948 [==============================] - 0s - loss: 10.6556 - acc: 0.3389 - val_loss: 10.1337 - val_acc: 0.3694
Epoch 6/30
3948/3948 [==============================] - 0s - loss: 10.6556 - acc: 0.3389 - val_loss: 10.1337 - val_acc: 0.3694
Epoch 7/30
3948/3948 [==============================] - 0s - loss: 10.6556 - acc: 0.3389 - val_loss: 10.1337 - val_acc: 0.3694
Epoch 8/30
3948/3948 [==============================] - 0s - loss: 10.6556 - acc: 0.3389 - val_loss: 10.1337 - val_acc: 0.

In [39]:
mocap_rot = []

for ses_mod in data2:
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))  
    x_rot[np.isnan(x_rot)]=0
    mocap_rot.append( x_rot )
    
mocap_rot = np.array(mocap_rot)
mocap_rot.shape

(4936, 200, 165)

In [41]:
def lstm_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(200, 165)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [42]:
model = lstm_model()
model.summary()

hist = model.fit(mocap_rot, Y, 
                 batch_size=100, nb_epoch=30, verbose=1, shuffle = True, 
                 validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 200, 512)          1388544   
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_23 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_23 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 2052      
_________________________________________________________________
activation_24 (Activation)   (None, 4)                 0         
Total params: 2,309,636
Trainable params: 2,309,636
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 31s - loss: 1.3738 - acc: 0.3138 - val_loss: 1.3631 - val_acc: 0.3026
Epoch 2/30
3948/3948 [==============================] - 30s - loss: 1.3583 - acc: 0.3300 - val_loss: 1.3247 - val_acc: 0.3694
Epoch 3/30
3948/3948 [==============================] - 30s - loss: 1.3411 - acc: 0.3381 - val_loss: 1.3363 - val_acc: 0.3694
Epoch 4/30
3948/3948 [==============================] - 30s - loss: 1.3323 - acc: 0.3333 - val_loss: 1.3459 - val_acc: 0.3573
Epoch 5/30
3948/3948 [==============================] - 30s - loss: 1.3308 - acc: 0.3488 - val_loss: 1.3349 - val_acc: 0.2915
Epoch 6/30
3948/3948 [==============================] - 30s - loss: 1.3299 - acc: 0.3430 - val_loss: 1.3339 - val_acc: 0.3522
Epoch 7/30
3948/3948 [==============================] - 30s - loss: 1.3098 - acc: 0.3696 - val_loss: 1.5831 - val_acc: 0.2966
Epoch 8/30
3948/3948 [==============================] - 30s - loss: 1.3

In [43]:

hist = model.fit(mocap_rot, Y, 
                 batch_size=100, nb_epoch=15, verbose=1, shuffle = True, 
                 validation_split=0.2)

Train on 3948 samples, validate on 988 samples
Epoch 1/15


/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


3948/3948 [==============================] - 30s - loss: 1.1814 - acc: 0.4473 - val_loss: 1.2605 - val_acc: 0.4079
Epoch 2/15
3948/3948 [==============================] - 30s - loss: 1.1952 - acc: 0.4402 - val_loss: 1.4706 - val_acc: 0.3128
Epoch 3/15
3948/3948 [==============================] - 30s - loss: 1.1538 - acc: 0.4709 - val_loss: 1.2523 - val_acc: 0.3937
Epoch 4/15
3948/3948 [==============================] - 30s - loss: 1.1612 - acc: 0.4633 - val_loss: 1.4993 - val_acc: 0.3381
Epoch 5/15
3948/3948 [==============================] - 30s - loss: 1.1688 - acc: 0.4694 - val_loss: 1.3648 - val_acc: 0.4221
Epoch 6/15
3948/3948 [==============================] - 30s - loss: 1.1780 - acc: 0.4663 - val_loss: 1.2808 - val_acc: 0.3998
Epoch 7/15
3948/3948 [==============================] - 30s - loss: 1.1479 - acc: 0.4739 - val_loss: 1.2455 - val_acc: 0.4514
Epoch 8/15
3948/3948 [==============================] - 30s - loss: 1.1294 - acc: 0.4742 - val_loss: 1.5512 - val_acc: 0.3907
Epo

In [46]:
def simple_conv(optimizer='Adam'):
    model = Sequential()
    model.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 165, 1)))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [53]:
model = simple_conv()
model.summary()

mocap_rot_re = mocap_rot.reshape(-1,200,165,1)

hist = model.fit(mocap_rot_re, Y, 
                 batch_size=100, nb_epoch=50, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, input_shape=(200, 165,..., padding="same", strides=(2, 2))`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", strides=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 100, 83, 32)       320       
_________________________________________________________________
dropout_22 (Dropout)         (None, 100, 83, 32)       0         
_________________________________________________________________
activation_46 (Activation)   (None, 100, 83, 32)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 50, 42, 64)        18496     
_________________________________________________________________
dropout_23 (Dropout)         (None, 50, 42, 64)        0         
_________________________________________________________________
activation_47 (Activation)   (None, 50, 42, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 25, 21, 64)        36928     
__________

In [55]:
def simple_conv2(optimizer='SGD'):
    model = Sequential()
    model.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 165, 1)))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [56]:
model = simple_conv2()
model.summary()

mocap_rot_re = mocap_rot.reshape(-1,200,165,1)

hist = model.fit(mocap_rot_re, Y, 
                 batch_size=100, nb_epoch=50, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, input_shape=(200, 165,..., padding="same", strides=(2, 2))`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", strides=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn(

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 100, 83, 32)       320       
_________________________________________________________________
dropout_29 (Dropout)         (None, 100, 83, 32)       0         
_________________________________________________________________
activation_53 (Activation)   (None, 100, 83, 32)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 50, 42, 64)        18496     
_________________________________________________________________
dropout_30 (Dropout)         (None, 50, 42, 64)        0         
_________________________________________________________________
activation_54 (Activation)   (None, 50, 42, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 25, 21, 64)        36928     
__________

3948/3948 [==============================] - 6s - loss: 1.1454 - acc: 0.4688 - val_loss: 1.4471 - val_acc: 0.4079
Epoch 43/50
3948/3948 [==============================] - 6s - loss: 1.1270 - acc: 0.4807 - val_loss: 1.4061 - val_acc: 0.4393
Epoch 44/50
3948/3948 [==============================] - 6s - loss: 1.1202 - acc: 0.4891 - val_loss: 1.7763 - val_acc: 0.3937
Epoch 45/50
3948/3948 [==============================] - 6s - loss: 1.1185 - acc: 0.4871 - val_loss: 1.3527 - val_acc: 0.4038
Epoch 46/50
3948/3948 [==============================] - 6s - loss: 1.1254 - acc: 0.4777 - val_loss: 1.5836 - val_acc: 0.4079
Epoch 47/50
3948/3948 [==============================] - 6s - loss: 1.1094 - acc: 0.4848 - val_loss: 1.3795 - val_acc: 0.3836
Epoch 48/50
3948/3948 [==============================] - 6s - loss: 1.1163 - acc: 0.4845 - val_loss: 1.4710 - val_acc: 0.3927
Epoch 49/50
3948/3948 [==============================] - 6s - loss: 1.1077 - acc: 0.4843 - val_loss: 1.6064 - val_acc: 0.4130
Epoc

In [57]:
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(4936, 200, 189, 1)

In [60]:
def simple_conv3(optimizer='SGD'):
    model = Sequential()
    model.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [61]:
model = simple_conv3()
model.summary()


hist = model.fit(x_train_mocap, Y, 
                 batch_size=100, nb_epoch=50, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, input_shape=(200, 189,..., padding="same", strides=(2, 2))`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", strides=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn(

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 100, 95, 32)       320       
_________________________________________________________________
dropout_41 (Dropout)         (None, 100, 95, 32)       0         
_________________________________________________________________
activation_66 (Activation)   (None, 100, 95, 32)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 50, 48, 64)        18496     
_________________________________________________________________
dropout_42 (Dropout)         (None, 50, 48, 64)        0         
_________________________________________________________________
activation_67 (Activation)   (None, 50, 48, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 25, 24, 64)        36928     
__________